## Problem

In [ ]:
def dy_dx(x):
  return 2**x

dy_dx(3) # But what if we increase the difficulty

8

In [ ]:
import math
# y = x^2, z = cos(y)
def dz_dx(x):
  return 2*x*math.cos(x**2)

dz_dx(2)

-2.6145744834544478

In [ ]:
# but what if y = x^2, z = sin y, u = e^2
# du/dx = du/dz dz/dy dy/dx -> code
# as the nested functions get complex their derivative calculations and coding gets complex

## AutoGrad

### Example 1

In [ ]:
import torch

x = torch.tensor(3.0, requires_grad=True) # use requires_grad=True whenever you want a gradient of a tensor
x

tensor(3., requires_grad=True)

In [ ]:
y = x**2

In [ ]:
y # grad_fn=<PowBackward0> store it as memory for backward pass to remember what operation was performed
# internal computation graph (x)---(sq/power)---(y)

tensor(9., grad_fn=<PowBackward0>)

In [ ]:
y.backward() # it will calculate the derivatives

In [ ]:
x.grad # to see it

tensor(6.)

### Example 2

In [ ]:
# now lets take y = x2, z = sin(y)
# we want to calculae dz/dx on a given value of x

x = torch.tensor(3.0, requires_grad=True)
y = x ** 2
z = torch.sin(y)

In [ ]:
x

tensor(3., requires_grad=True)

In [ ]:
y

tensor(9., grad_fn=<PowBackward0>)

In [ ]:
z

tensor(0.4121, grad_fn=<SinBackward0>)

In [ ]:
# computation graph
# (x)---(sq)---(y)---(sin)---(z)
# we got z and y value but for gradient (dz/dx) we need to go backward
# first dz/dy and then dy/dx

z.backward()

In [ ]:
x.grad # x is leaf node only this gets calculated not intermediate/root nodes like y and z

tensor(-5.4668)

# Training Pipeline

In [ ]:
# 1. Load the dataset
# 2. Basic preprocessing
# 3. Training Process
    # i) Create the model
    # ii) Forward Pass
    # iii) Loss Calculation
    # iv) Backprop
    # v) Parameters update
# 4. Model evaluation

In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [6]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [7]:
df.shape

(569, 33)

In [9]:
df.drop(columns=['id', 'Unnamed: 32'], inplace=True)

In [10]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


Train Test Split

In [11]:
x_train, x_test, y_train, y_test = train_test_split(df.iloc[:,1:], df.iloc[:,0], test_size=0.2)

Scaling

In [12]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [13]:
x_train

array([[-0.48071852,  1.83441418, -0.41958942, ...,  0.50572239,
         0.55586062,  2.07545801],
       [-1.21642721, -0.7907996 , -1.22446144, ..., -0.93238953,
        -0.06791233, -0.11165258],
       [-1.08672953, -0.36616717, -1.07905199, ..., -1.17454006,
        -0.02288059, -0.3182287 ],
       ...,
       [ 1.77058404, -1.10282233,  1.79022461, ...,  0.75885204,
        -0.86347303, -0.39216715],
       [ 1.73943394,  0.11711616,  1.73697608, ...,  1.59448488,
         0.25064551, -0.16809414],
       [-0.56000968, -1.47818802, -0.59612878, ..., -0.6478474 ,
        -0.61329671, -0.26799571]])

In [14]:
y_train

,diagnosis
193,M
273,B
217,B
417,M
459,B
...,...
126,M
340,B
4,M
432,M


Label Encoder

In [15]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [16]:
y_train

array([1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1,

Numpy arrays to PyTorch tensors

In [17]:
x_train_tensor = torch.from_numpy(x_train)
x_test_tensor = torch.from_numpy(x_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [18]:
x_train_tensor.shape

torch.Size([455, 30])

In [19]:
y_train_tensor.shape

torch.Size([455])

# Defining the model

In [29]:
class MySimpleNN():
  def __init__(self, x):
    self.weights = torch.rand(x.shape[1], 1, dtype=torch.float64, requires_grad=True) # x.shape[1] = 30, (features on dataset shate 0 = 455, 1 = 30 in x)
    self.bias = torch.zeros(1, dtype=torch.float64, requires_grad=True)

  def forward(self, x):
    z = torch.matmul(x, self.weights) + self.bias # z = wx + b
    y_pred = torch.sigmoid(z)
    return y_pred
  def loss_function(self, y_pred, y):
    # clamp predictions to avoid log(0)
    epsilon = 1e-7
    y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)

    # calculate loss
    loss = -(y_train_tensor + torch.log(y_pred) + (1 - y_train_tensor) + torch.log(1 - y_pred)).mean() # cross entropy formula
    return loss

Important Parameter

In [34]:
learning_rate = 0.1
epochs = 25

# Training Pipeline

In [36]:
# create model
model = MySimpleNN(x_train_tensor)
# define loop
for epoch in range(epochs):
  # forward pass (wx+b) sigmoid on z
  y_pred = model.forward(x_train_tensor)

  # loss calculate
  loss = model.loss_function(y_pred, y_train_tensor)
  # backward pass
  loss.backward()

  # parameters update
  with torch.no_grad():
  # W new = W old - learning-rate (del L/ del x)
    model.weights -= learning_rate * model.weights.grad
  # b new = b old - learning rate (del L / del b)
    model.bias -= learning_rate * model.bias.grad

  # zero gradients # gradients accumulate but we dont want that so we want it start from zero everytime
  model.weights.grad.zero_()
  model.bias.grad.zero_()

  # print loss in each epoch
  print(f"Epoch: {epoch + 1}, loss: {loss.item()}")

Epoch: 1, loss: 5.880883697435499
Epoch: 2, loss: 5.276686350286992
Epoch: 3, loss: 4.640972466895965
Epoch: 4, loss: 3.985286884317111
Epoch: 5, loss: 3.2937415930629266
Epoch: 6, loss: 2.6204025648885207
Epoch: 7, loss: 1.9753448904570625
Epoch: 8, loss: 1.4126121949731154
Epoch: 9, loss: 0.9918530968672417
Epoch: 10, loss: 0.7328932725258723
Epoch: 11, loss: 0.6067814416954884
Epoch: 12, loss: 0.5502998133789051
Epoch: 13, loss: 0.5206249594351551
Epoch: 14, loss: 0.5021317903428066
Epoch: 15, loss: 0.48950299893644766
Epoch: 16, loss: 0.48039777499264
Epoch: 17, loss: 0.47353142379339264
Epoch: 18, loss: 0.4681296555554746
Epoch: 19, loss: 0.4637116045370878
Epoch: 20, loss: 0.4599747235297338
Epoch: 21, loss: 0.4567261642180318
Epoch: 22, loss: 0.4538407922857496
Epoch: 23, loss: 0.4512354813691804
Epoch: 24, loss: 0.44885341271600865
Epoch: 25, loss: 0.446654485911381


In [37]:
model.weights

tensor([[ 0.1855],
        [-0.1383],
        [ 0.4543],
        [-0.0571],
        [-0.3227],
        [-0.2327],
        [-0.0867],
        [-0.1621],
        [ 0.0564],
        [ 0.3679],
        [-0.0403],
        [-0.0820],
        [-0.3142],
        [ 0.2174],
        [ 0.2957],
        [-0.1116],
        [ 0.3683],
        [-0.2201],
        [ 0.0186],
        [ 0.1107],
        [ 0.0862],
        [ 0.2223],
        [-0.4073],
        [ 0.2115],
        [ 0.3144],
        [-0.2140],
        [ 0.2289],
        [ 0.0996],
        [ 0.1034],
        [-0.3192]], dtype=torch.float64, requires_grad=True)

In [38]:
model.bias

tensor([0.0657], dtype=torch.float64, requires_grad=True)

Evaluation

In [50]:
# model evaluation
with torch.no_grad():
  y_pred = model.forward(x_test_tensor)
  y_pred = (y_pred > 0.9).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f"Accuracy: {accuracy.item()}")

Accuracy: 0.5592489838600159
